# Column Statistics Analysis - Step 1: PCDS Data Collection

This notebook collects column-level statistics from the PCDS (Oracle) platform.
It will save the results for use in step 3 (comparison).

## Cell 1: Import Required Libraries

In [ ]:
import re
import os
import csv
import json
import pickle
import shutil
import argparse
import warnings
import numpy as np
import pandas as pd
import datetime as dt
import time

from upath import UPath
from loguru import logger
from tqdm import tqdm
from typing import Literal
from dataclasses import dataclass, field, fields

warnings.filterwarnings('ignore', message=r'pandas only supports SQLAlchemy connectable .*', category=UserWarning)

# Note: This notebook uses Parquet format for cross-platform compatibility
# Install pyarrow if needed: pip install pyarrow or conda install -c conda-forge pyarrow
import pyarrow

## Cell 2: Constants and Configuration

In [ ]:
# --- Global Constants ---
SEP = '; '
TODAY = dt.datetime.now()
WIDTH = 80
inWindows = os.name == 'nt'

TPartition = Literal['whole', 'year', 'year_month', 'empty', 'year_week', 'week', 'snapshot']

# --- SQL Template for PCDS Column Statistics ---
PCDS_SQL_COLUMN = """
DECLARE
    v_sql          VARCHAR2(20000);
    v_col_name     VARCHAR2(128);
    v_data_type    VARCHAR2(128);
    v_table_name   VARCHAR2(128) := UPPER('{table}');

    v_ret_data_type  VARCHAR2(128);
    v_count          NUMBER;
    v_distinct       NUMBER;
    v_max            VARCHAR2(4000);
    v_min            VARCHAR2(4000);
    v_missing        NUMBER;
    v_avg            NUMBER;
    v_std            NUMBER;
    v_sum            NUMBER;
    v_sum_sq         NUMBER;
    v_freq           VARCHAR2(4000);

    l_column_ref VARCHAR2(256);

BEGIN
    FOR rec IN (
        SELECT column_name, data_type
        FROM all_tab_cols
        WHERE table_name = v_table_name
        ORDER BY column_id
    ) LOOP
        v_col_name := rec.column_name;
        v_data_type := rec.data_type;

        v_ret_data_type := NULL; v_count := NULL; v_distinct := NULL; 
        v_max := NULL; v_min := NULL; v_missing := NULL; 
        v_avg := NULL; v_std := NULL; v_sum := NULL; v_sum_sq := NULL; v_freq := NULL;
        
        IF v_data_type LIKE 'TIMESTAMP%' THEN 
            l_column_ref := 'TRUNC(' || v_col_name || ')';
        ELSE
            l_column_ref := v_col_name;
        END IF;

        IF v_data_type IN ('NUMBER', 'FLOAT', 'BINARY_FLOAT', 'BINARY_DOUBLE') THEN
            v_sql := 'SELECT ';
            v_sql := v_sql || '''' || v_data_type || ''' AS col_type, ';
            v_sql := v_sql || 'COUNT(' || v_col_name || ') AS col_count, ';
            v_sql := v_sql || 'COUNT(DISTINCT ' || v_col_name || ') AS col_distinct, ';
            v_sql := v_sql || 'MAX(' || v_col_name || ') AS col_max, ';
            v_sql := v_sql || 'MIN(' || v_col_name || ') AS col_min, ';
            v_sql := v_sql || 'AVG(' || v_col_name || ') AS col_avg, ';
            v_sql := v_sql || 'STDDEV_SAMP(' || v_col_name || ') AS col_std, ';
            v_sql := v_sql || 'SUM(' || v_col_name || ') AS col_sum, ';
            v_sql := v_sql || 'SUM(' || v_col_name || ' * ' || v_col_name || ') AS col_sum_sq, ';
            v_sql := v_sql || 'COUNT(*) - COUNT(' || v_col_name || ') AS col_missing, ';
            v_sql := v_sql || 'EMPTY_CLOB() AS col_freq ';
            v_sql := v_sql || 'FROM ' || v_table_name || ' WHERE {limit} ';
        ELSE
            v_sql := 'WITH FreqTable_RAW AS ( ';
            v_sql := v_sql || 'SELECT ' || l_column_ref || ' AS p_col, COUNT(*) AS value_freq '; 
            v_sql := v_sql || 'FROM ' || v_table_name || ' WHERE {limit} '; 
            v_sql := v_sql || 'GROUP BY ' || l_column_ref || ' '; 
            v_sql := v_sql || '), FreqTable AS ( '; 
            v_sql := v_sql || 'SELECT p_col, value_freq, '; 
            v_sql := v_sql || 'ROW_NUMBER() OVER (ORDER BY value_freq DESC, p_col ASC) AS rn '; 
            v_sql := v_sql || 'FROM FreqTable_RAW), AggStats AS ( ';
            v_sql := v_sql || 'SELECT SUM(ft.value_freq) AS col_count, '; 
            v_sql := v_sql || 'COUNT(ft.value_freq) AS col_distinct, '; 
            v_sql := v_sql || 'MAX(ft.value_freq) AS col_max, '; 
            v_sql := v_sql || 'MIN(ft.value_freq) AS col_min, '; 
            v_sql := v_sql || 'AVG(ft.value_freq) AS col_avg, '; 
            v_sql := v_sql || 'STDDEV_SAMP(ft.value_freq) AS col_std, ';
            v_sql := v_sql || 'SUM(ft.value_freq) AS col_sum, ';
            v_sql := v_sql || 'SUM(ft.value_freq * ft.value_freq) AS col_sum_sq '; 
            v_sql := v_sql || 'FROM FreqTable ft) SELECT ';
            v_sql := v_sql || '''' || v_data_type || ''' AS col_type, ast.*, '; 
            v_sql := v_sql || '(SELECT NVL(value_freq, 0) FROM FreqTable WHERE p_col IS NULL) AS col_missing, '; 
            v_sql := v_sql || '(SELECT LISTAGG(p_col || ''('' || value_freq || '')'', ''; '') WITHIN GROUP (ORDER BY value_freq DESC) FROM FreqTable WHERE rn <= 10) AS col_freq ';
            v_sql := v_sql || 'FROM AggStats ast';
        END IF;

        EXECUTE IMMEDIATE v_sql INTO
           v_ret_data_type, v_count, v_distinct, v_max, v_min, 
           v_avg, v_std, v_sum, v_sum_sq, v_missing, v_freq;

        DBMS_OUTPUT.PUT_LINE('Column: ' || v_col_name);
        DBMS_OUTPUT.PUT_LINE('  col_type: ' || v_ret_data_type); 
        DBMS_OUTPUT.PUT_LINE('  col_count: ' || v_count);
        DBMS_OUTPUT.PUT_LINE('  col_distinct: ' || v_distinct);
        DBMS_OUTPUT.PUT_LINE('  col_max: ' || v_max);
        DBMS_OUTPUT.PUT_LINE('  col_min: ' || v_min);
        DBMS_OUTPUT.PUT_LINE('  col_avg: ' || v_avg);
        DBMS_OUTPUT.PUT_LINE('  col_std: ' || v_std);
        DBMS_OUTPUT.PUT_LINE('  col_sum: ' || v_sum);
        DBMS_OUTPUT.PUT_LINE('  col_sum_sq: ' || v_sum_sq);
        DBMS_OUTPUT.PUT_LINE('  col_freq: ' || v_freq);
        DBMS_OUTPUT.PUT_LINE('  col_missing: ' || v_missing);
        DBMS_OUTPUT.PUT_LINE('---');
    END LOOP;
END;
"""

## Cell 3: Core Data Types and Classes

In [ ]:
class Timer:
    """Context manager for timing code execution"""
    
    def __enter__(self):
        self.start = time.perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        pass

    @property
    def time(self):
        return time.perf_counter() - self.start
    
    def pause(self):
        """Return elapsed time and reset timer"""
        elapsed = self.time
        self.start = time.perf_counter()
        return elapsed

    @staticmethod
    def to_str(value):
        """Convert seconds to human-readable format"""
        minutes, seconds = divmod(value, 60)
        hours, minutes = divmod(minutes, 60)
        return f'{hours} hours {minutes} minutes {seconds:.0f} seconds'

@dataclass
class MetaOut:
    """Metadata output structure"""
    col2COL: dict
    col2type: dict
    infostr: str
    rowvar: str
    rowexclude: list
    rowtype: str
    nrows: int
    where: str

@dataclass(init=False)
class MetaJSON:
    """Container for metadata from previous meta analysis step"""
    pcds: MetaOut
    aws: MetaOut
    last_modified: str
    partition: TPartition = 'whole'
    tokenised_cols: list = field(default_factory=list)

    def __init__(self, **kwargs):
        field_names = [f.name for f in fields(self)]
        for k, v in kwargs.items():
            if k in field_names:
                setattr(self, k, v)
        
        def col2col(a_str, b_str, sep=SEP):
            return {k: v for k, v in zip(a_str.split(sep), b_str.split(sep))}
        
        for key, other in [('pcds', 'aws'), ('aws', 'pcds')]:
            out = MetaOut(
                rowvar=kwargs['%s_dt' % key],
                infostr=kwargs['%s_tbl' % key],
                where=kwargs['%s_where' % key],
                nrows=kwargs['%s_nrows' % key],
                col2COL=col2col(kwargs['%s_cols' % key], kwargs['%s_cols' % other]),
                col2type=col2col(kwargs['%s_cols' % key], kwargs['%s_types' % key]),
                rowtype=kwargs['%s_dt_type' % key],
                rowexclude=kwargs['%s_exclude' % key]
            )
            setattr(self, key, out)

@dataclass
class CSMeta:
    """Metadata for column statistics comparison"""
    pcds_table: str
    aws_table: str
    partition: TPartition
    vintage: str
    pcds_time: int
    aws_time: int = 0

    def todict(self):
        return {f.name: getattr(self, f.name) for f in fields(self)}

## Cell 4: Utility Functions

In [ ]:
def start_run():
    logger.info('\n\n' + '=' * WIDTH)

def end_run():
    logger.info('\n\n' + '=' * WIDTH)

def load_env(file):
    if inWindows:
        from dotenv import load_dotenv
        load_dotenv(file)

class IO:
    """File I/O utilities - uses Parquet/JSON for cross-platform compatibility"""

    @staticmethod
    def write_dataframe(file, df):
        """Save DataFrame in portable Parquet format"""
        file = UPath(file)
        df.to_parquet(file, index=True, engine='pyarrow', compression='snappy')

    @staticmethod
    def read_dataframe(file):
        """Load DataFrame from Parquet format"""
        file = UPath(file)
        return pd.read_parquet(file, engine='pyarrow')

    @staticmethod
    def write_json(file, data, cls=None):
        """Save to JSON with proper serialization"""
        import numpy as np
        import pandas as pd
        import datetime as dt

        def convert(obj):
            if isinstance(obj, (np.integer, np.floating)):
                return obj.item()
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif pd.isna(obj):
                return None
            elif isinstance(obj, (dt.datetime, dt.date)):
                return obj.isoformat()
            elif isinstance(obj, set):
                return list(obj)
            raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

        with open(file, 'w') as f:
            json.dump(data, f, indent=2, default=convert, cls=cls)

    @staticmethod
    def read_json(file):
        """Load from JSON"""
        with open(file, 'r') as f:
            return json.load(f)

    @staticmethod
    def write_pickle(file, data):
        """Deprecated: Use write_dataframe or write_json instead"""
        with open(file, 'wb') as f:
            pickle.dump(data, f)

    @staticmethod
    def read_pickle(file):
        """Deprecated: Use read_dataframe or read_json instead"""
        with open(file, 'rb') as f:
            return pickle.load(f)

    @staticmethod
    def read_meta_json(json_file):
        """Read metadata JSON and convert to MetaJSON objects"""
        data = IO.read_json(json_file)
        return {k: MetaJSON(**v) for k, v in data.items()}

    @staticmethod
    def delete_file(file):
        if (filepath := UPath(file)).exists():
            filepath.unlink()

## Cell 5: Date Handling Functions for PCDS

In [ ]:
def get_iso_week_dates(year, week):
    jan01, dec31 = dt.datetime(year, 1, 1), dt.datetime(year, 12, 31)
    first_day = jan01 - dt.timedelta(days=jan01.weekday())
    start = first_day + dt.timedelta(weeks=week - 1)
    end = start + dt.timedelta(days=6)
    start, end = max(start, jan01), min(end, dec31)
    return start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')

def parse_format_date(str_w_format):
    pattern = r'^(.+?)(?:\s*\(([^)]+)\))?$'
    return re.match(pattern, str_w_format)

def parse_exclude_date(exclude_clause):
    """Convert date exclusions to PCDS format"""
    p1 = r"TO_CHAR\((?P<col>\w+),\s*'YYYY-MM-DD'\)\s+(?P<op>not in|in)\s+\((?P<dates>.*?)\)"
    if m := re.match(p1, exclude_clause, flags=re.I):
        col, op, dates = m.groups()
        new_dates = ', '.join(f"DATE {date.strip()}" for date in dates.split(','))
        return '%s %s (%s)' % (col, op, new_dates)
    return exclude_clause

def get_pcds_where(date_var, date_type, date_partition, date_range, date_format, snapshot=None, exclude_clauses=[]):
    if date_type and ('char' in date_type.lower() or 'varchar' in date_type.lower()):
        date_var = f"TO_DATE({date_var}, '{date_format}')"
    
    if snapshot:
        return ' AND '.join(parse_exclude_date(x) for x in exclude_clauses if x)
    elif date_partition == 'whole':
        base_clause = "1=1"
    elif date_partition == 'year':
        start_dt = f"TO_DATE('{date_range}-01-01', 'YYYY-MM-DD')"
        end_dt = f"TO_DATE('{date_range}-12-31', 'YYYY-MM-DD')"
        base_clause = f"{date_var} >= {start_dt} AND {date_var} <= {end_dt}"
    elif date_partition == 'year_month':
        start_dt = f"TO_DATE('{date_range}', 'YYYY-MM')"
        end_dt = f"LAST_DAY(TO_DATE('{date_range}', 'YYYY-MM'))"
        base_clause = f"{date_var} >= {start_dt} AND {date_var} <= {end_dt}"
    elif date_partition in ('year_week', 'week'):
        year, week = date_range.split('-W')
        start_dt, end_dt = get_iso_week_dates(int(year), int(week))
        base_clause = f"{date_var} >= DATE '{start_dt}' AND {date_var} <= DATE '{end_dt}'"
    elif date_partition == 'daily':
        target_dt = f"TO_DATE('{date_range}', 'YYYY-MM-DD')"
        base_clause = f"{date_var} = {target_dt}"
    else:
        raise ValueError(f"Unsupported partition type: {date_partition}")
    
    if (exclude_clauses := [x for x in exclude_clauses if x]):
        exclude_clause = ' AND '.join(parse_exclude_date(x) for x in exclude_clauses if x)
        return f"({base_clause}) AND ({exclude_clause})"
    else:
        return base_clause

## Cell 6: PCDS SQL Engine

In [ ]:
import pandas.io.sql as psql

class SQLengine:
    """SQL query engine for PCDS"""
    
    def __init__(self):
        self.reset()

    def reset(self):
        self._where = None
        self._type = None
        self._date = None
        self._dateraw = None
        self._table = None

    def query(self, query, connection, **query_kwargs):
        """Execute SQL query and return DataFrame"""
        df = psql.read_sql_query(query, connection, **query_kwargs)
        df.columns = [x.upper() for x in df.columns]
        return df

    def execute_PCDS(self, query, service_name):
        """Execute PCDS PL/SQL block and parse DBMS_OUTPUT"""
        from oracledb import STRING, NUMBER
        import oracledb
        
        query_stmt = query
        
        # TODO: Implement your PCDS connection logic here
        # with pcds_connect(service_name=service_name) as CONN:
        #     cursor = CONN.cursor()
        #     cursor.callproc("dbms_output.enable", [None])
        #     cursor.execute(query_stmt)
        #     ...
        raise NotImplementedError("Implement PCDS connection")

    def query_PCDS(self, query_stmt: str, service_name: str, **query_kwargs):
        """Execute query on PCDS"""
        # TODO: Implement your PCDS connection logic here
        # with pcds_connect(service_name=service_name) as CONN:
        #     return self.query(query_stmt, CONN, **query_kwargs)
        raise NotImplementedError("Implement PCDS connection")

proc_pcds = SQLengine()

## Cell 7: PCDS Column Analyzer

In [ ]:
class ColumnAnalyzer:
    """PCDS column analysis engine"""
    
    def run_pcds_column_analysis(self, info_str: str, where_clause: str = "1=1") -> pd.DataFrame:
        """Run comprehensive column statistics on PCDS table"""
        try:
            service, table_name = info_str.split('.')
            where_clause = where_clause.replace("'", "''")
            sql_stmt = PCDS_SQL_COLUMN.format(table=table_name, limit=where_clause)
            logger.info(f"Executing PCDS analysis for {table_name}")
            result = proc_pcds.execute_PCDS(sql_stmt, service_name=service)
            return pd.DataFrame(result).T
        except Exception as e:
            logger.error(f"Error in PCDS analysis: {e}")
            raise

    def download_pcds_snapshot(self, info_str, columns, where_clause) -> pd.DataFrame:
        """Download PCDS table snapshot for backup"""
        try:
            service, table_name = info_str.split('.')
            columns = ', '.join(columns)
            sql_stmt = f"SELECT {columns} FROM {table_name} WHERE {where_clause}"
            logger.info(f"Downloading PCDS table {table_name}")
            return proc_pcds.query_PCDS(sql_stmt, service_name=service)
        except Exception as e:
            logger.error(f"Error in PCDS pulling: {e}")
            raise

## Cell 8: Main Execution - PCDS Data Collection

In [ ]:
def main_pcds():
    """Main execution function for PCDS column statistics collection"""

    # Configuration - adjust these paths as needed
    meta_json_path = 'path/to/meta_analysis_output.json'  # From meta_analysis step
    meta_csv_path = 'path/to/meta_analysis.csv'
    output_folder = UPath('output/column_stats_pcds')
    output_folder.mkdir(exist_ok=True, parents=True)

    start_run()

    # Load metadata from previous meta analysis
    meta_json = IO.read_meta_json(meta_json_path)
    meta_csv = pd.read_csv(meta_csv_path)

    CA = ColumnAnalyzer()
    summary_data = {}

    for i, row in tqdm(meta_csv.iterrows(), desc='Processing PCDS tables...', total=len(meta_csv)):
        name = row.get('PCDS Table Details with DB Name')
        logger.info(f"Processing dataset: {name}")

        # Load metadata for this table
        meta_info = meta_json.get(name)
        if not meta_info:
            continue

        meta_pcds = meta_info.pcds
        partition = meta_info.partition

        if partition == 'empty':
            continue

        # TODO: Implement vintage determination logic
        vintages = ['entire_dataset']  # Placeholder

        # Clean table name for file naming
        table_name = name.split('.')[-1].lower()

        for vintage in vintages:
            logger.info(f"Processing vintage: {vintage}")

            # Build WHERE clause for this vintage
            pcds_where = get_pcds_where(
                date_var=meta_pcds.rowvar,
                date_type=meta_pcds.rowtype,
                date_partition=partition,
                date_range=vintage,
                date_format='YYYY-MM-DD',
                snapshot=partition == 'snapshot',
                exclude_clauses=[meta_pcds.where, meta_pcds.rowexclude]
            )

            # Compute column statistics
            with Timer() as timer:
                pcds_stats = CA.run_pcds_column_analysis(
                    meta_pcds.infostr, pcds_where
                )
                pcds_time = timer.pause()

            # Save individual parquet file for this table/vintage
            stats_file = output_folder / f'pcds_stats_{table_name}_{vintage}.parquet'
            IO.write_dataframe(stats_file, pcds_stats)
            logger.info(f"Saved stats to {stats_file}")

            # Save metadata as JSON
            meta_data = CSMeta(
                pcds_table=meta_pcds.infostr,
                aws_table=meta_info.aws.infostr,
                partition=partition,
                vintage=vintage,
                pcds_time=pcds_time,
            ).todict()

            meta_file = output_folder / f'pcds_meta_{table_name}_{vintage}.json'
            IO.write_json(meta_file, {
                'meta_data': meta_data,
                'pcds_where': pcds_where
            })
            logger.info(f"Saved metadata to {meta_file}")

            # Add to summary
            if name not in summary_data:
                summary_data[name] = {}
            summary_data[name][vintage] = {
                'stats_file': str(stats_file),
                'meta_file': str(meta_file),
                'table_name': table_name,
                'meta_data': meta_data
            }

            logger.info(f"Completed PCDS analysis for vintage {vintage}")

        proc_pcds.reset()

    # Save summary file
    summary_file = output_folder / 'pcds_summary.json'
    IO.write_json(summary_file, summary_data)
    logger.info(f"Summary saved to {summary_file}")
    logger.info(f"Processed {len(summary_data)} tables with {sum(len(v) for v in summary_data.values())} total vintages")

    end_run()
    return summary_data

if __name__ == '__main__':
    results = main_pcds()

## Run the Analysis

Uncomment the cell below to run the PCDS data collection:

In [ ]:
# results = main_pcds()